In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [2]:
# Reading in Data
acled = pd.read_csv('data/ACLED.csv')
acled.head()

,data_id,iso,event_id_cnty,event_id_no_cnty,event_date,year,time_precision,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,source,source_scale,notes,fatalities,timestamp,iso3
0,6708242,368,IRQ22736,22736,31-Dec-19,2019,1,Protests,Peaceful protest,Protesters (Iraq),NaN,6,NaN,NaN,0,60,Middle East,Iraq,Basrah,Al-Qurna,NaN,Qurna,31.0167,47.4333,1,Al Mirbad,National,"On Dec 31, protesters in Qurna came out to den...",0,1578503874,IRQ
1,6716953,760,SYR73663,73663,31-Dec-19,2019,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Syria (2000-),NaN,1,NaN,NaN,0,10,Middle East,Syria,Idleb,Al Ma'ra,Kafr Nobol,Hazarin,35.5998,36.5267,1,SOHR,Other,"On 31 December 2019, regime forces shelled Haz...",0,1578515228,SYR
2,6716955,760,SYR73713,73713,31-Dec-19,2019,1,Explosions/Remote violence,Shelling/artillery/missile attack,Opposition Rebels (Syria),NaN,2,Military Forces of Syria (2000-),NaN,1,12,Middle East,Syria,Hama,Masyaf,Jeb Ramleh,Aslieh,35.1967,36.4892,1,SOHR,Other,"On 31 December 2019, opposition rebels shelled...",0,1578515228,SYR
3,6716700,760,SYR73184,73184,31-Dec-19,2019,1,Battles,Armed clash,Unidentified Armed Group (Syria),NaN,3,Opposition Rebels (Syria),NaN,2,23,Middle East,Syria,Aleppo,Al Bab,Ar-Ra'ee,Ar-Ra'ee,36.6125,37.4464,1,SOHR,Other,"On 31 December 2019, unidentified gunmen assas...",1,1578515227,SYR
4,6716960,760,SYR73664,73664,31-Dec-19,2019,1,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Syria (2000-),NaN,1,NaN,NaN,0,10,Middle East,Syria,Idleb,Al Ma'ra,Kafr Nobol,Kafr Nobol,35.6147,36.5603,1,SOHR,Other,"On 31 December 2019, regime forces shelled Kaf...",0,1578515228,SYR


In [3]:
# Converting Date column to panas datetime format
acled['event_date'] = pd.to_datetime(acled['event_date'])
acled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91910 entries, 0 to 91909
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   data_id           91910 non-null  int64         
 1   iso               91910 non-null  int64         
 2   event_id_cnty     91910 non-null  object        
 3   event_id_no_cnty  91910 non-null  int64         
 4   event_date        91910 non-null  datetime64[ns]
 5   year              91910 non-null  int64         
 6   time_precision    91910 non-null  int64         
 7   event_type        91910 non-null  object        
 8   sub_event_type    91910 non-null  object        
 9   actor1            91910 non-null  object        
 10  assoc_actor_1     19824 non-null  object        
 11  inter1            91910 non-null  int64         
 12  actor2            55510 non-null  object        
 13  assoc_actor_2     9890 non-null   object        
 14  inter2            9191

In [4]:
# Dropping unecessary columns
acled.drop(columns=['data_id',
                    'iso',
                    'event_id_cnty',
                    'event_id_no_cnty',
                    'time_precision',
                    'source',
                    'source_scale',
                    'notes',
                    'iso3'
                    ],
                    inplace=True)

In [5]:
acled.head()

,event_date,year,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,fatalities,timestamp
0,2019-12-31,2019,Protests,Peaceful protest,Protesters (Iraq),NaN,6,NaN,NaN,0,60,Middle East,Iraq,Basrah,Al-Qurna,NaN,Qurna,31.0167,47.4333,1,0,1578503874
1,2019-12-31,2019,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Syria (2000-),NaN,1,NaN,NaN,0,10,Middle East,Syria,Idleb,Al Ma'ra,Kafr Nobol,Hazarin,35.5998,36.5267,1,0,1578515228
2,2019-12-31,2019,Explosions/Remote violence,Shelling/artillery/missile attack,Opposition Rebels (Syria),NaN,2,Military Forces of Syria (2000-),NaN,1,12,Middle East,Syria,Hama,Masyaf,Jeb Ramleh,Aslieh,35.1967,36.4892,1,0,1578515228
3,2019-12-31,2019,Battles,Armed clash,Unidentified Armed Group (Syria),NaN,3,Opposition Rebels (Syria),NaN,2,23,Middle East,Syria,Aleppo,Al Bab,Ar-Ra'ee,Ar-Ra'ee,36.6125,37.4464,1,1,1578515227
4,2019-12-31,2019,Explosions/Remote violence,Shelling/artillery/missile attack,Military Forces of Syria (2000-),NaN,1,NaN,NaN,0,10,Middle East,Syria,Idleb,Al Ma'ra,Kafr Nobol,Kafr Nobol,35.6147,36.5603,1,0,1578515228


In [6]:
acled.isnull().sum()

event_date            0
year                  0
event_type            0
sub_event_type        0
actor1                0
assoc_actor_1     72086
inter1                0
actor2            36400
assoc_actor_2     82020
inter2                0
interaction           0
region                0
country               0
admin1                0
admin2               21
admin3            20379
location              0
latitude              0
longitude             0
geo_precision         0
fatalities            0
timestamp             0
dtype: int64

In [7]:
# Filling unknown actors and locations with 'unknown'.

acled.fillna('unknown', inplace=True)

In [8]:
# Reading in Tor Data
tor_filepaths = [
    'data/Egypt-Tor-Metrics.csv',
    'data/Iraq-Tor-Metrics.csv',
    'data/Jordan-Tor-Metrics.csv',
    'data/Lebanon-Tor-Metrics.csv',
    'data/Saudi-Arabia-Tor-Metrics.csv',
    'data/Syria-Tor-Metrics.csv'
    ]

tor_data = [csv for csv in tor_filepaths]
tor_metrics = pd.concat(map(pd.read_csv, tor_filepaths))
tor_metrics['date'] = pd.to_datetime(tor_metrics['date'])

In [9]:
tor_metrics['country'].unique()

array(['eg', 'iq', 'jo', 'lb', 'sa', 'sy'], dtype=object)

In [10]:
# Mapping country names to match ACLED Data
tor_metrics['country'] = tor_metrics['country'].map({'eg': 'Egypt', 'iq': 'Iraq', 'jo':'Jordan', 'lb': 'Lebanon', 'sa': 'Saudi Arabia', 'sy': 'Syria'})

In [11]:
tor_metrics.head()

,date,country,users,frac
0,2017-01-01,Egypt,564,56
1,2017-01-02,Egypt,654,54
2,2017-01-03,Egypt,598,59
3,2017-01-04,Egypt,528,57
4,2017-01-05,Egypt,508,59


In [12]:
# Merging datasets
acled_and_tor = pd.merge_ordered(acled, 
                         tor_metrics, 
                         how='left',
                         left_on=['event_date', 'country'],
                         right_on=['date', 'country'],
                         fill_method='ffill')

In [13]:
# Filtering for US/UN Actions
us_actions = acled_and_tor[(acled_and_tor['actor1'].str.contains('United States|United Nations')) |
                           (acled_and_tor['actor2'].str.contains('United States|United Nations')) |
                           (acled_and_tor['assoc_actor_1'].str.contains('United States|United Nations')) |
                           (acled_and_tor['assoc_actor_2'].str.contains('United States|United Nations'))] 

In [14]:
# Dropping US/UN Actions
acled_and_tor.drop(us_actions.index, inplace=True)

In [15]:
acled_and_tor.isnull().sum()

event_date        0
year              0
event_type        0
sub_event_type    0
actor1            0
assoc_actor_1     0
inter1            0
actor2            0
assoc_actor_2     0
inter2            0
interaction       0
region            0
country           0
admin1            0
admin2            0
admin3            0
location          0
latitude          0
longitude         0
geo_precision     0
fatalities        0
timestamp         0
date              0
users             0
frac              0
dtype: int64

In [16]:
# Dropping data column for tor data
acled_and_tor.drop(columns='date', axis=1, inplace=True)

In [17]:
# Extracting Month Column and Creating Seperate Column
acled_and_tor['month'] = acled_and_tor['event_date'].dt.month

In [18]:
econ_data = pd.read_csv('data/econ-data.csv')

In [19]:
econ_data['Date'] = pd.to_datetime(econ_data['Date'])

In [20]:
conflict = pd.merge(acled_and_tor, econ_data,
                            how='left',
                            left_on=['country', 'year', 'month'],
                            right_on=['Country', 'Year', 'Month'])

In [21]:
conflict.isnull().sum()

event_date                                                      0
year                                                            0
event_type                                                      0
sub_event_type                                                  0
actor1                                                          0
assoc_actor_1                                                   0
inter1                                                          0
actor2                                                          0
assoc_actor_2                                                   0
inter2                                                          0
interaction                                                     0
region                                                          0
country                                                         0
admin1                                                          0
admin2                                                          0
admin3    

In [22]:
conflict['missing_data'] = conflict['Consumer Price Index (CPI)'].map({np.nan:1})
conflict['missing_data'].fillna(0, inplace=True)

In [23]:
conflict.drop(columns=econ_data.columns, inplace=True)

In [24]:
conflict.head()

,event_date,year,event_type,sub_event_type,actor1,assoc_actor_1,inter1,actor2,assoc_actor_2,inter2,interaction,region,country,admin1,admin2,admin3,location,latitude,longitude,geo_precision,fatalities,timestamp,users,frac,month,missing_data
0,2017-01-01,2017,Explosions/Remote violence,Remote explosive/landmine/IED,Unidentified Armed Group (Egypt),unknown,3,unknown,unknown,0,30,Northern Africa,Egypt,Cairo,Al Maadi,unknown,Cairo-Al Maadi,29.9667,31.2500,1,0,1578503454,564,56,1,0.0
1,2017-01-01,2017,Battles,Armed clash,Military Forces of Iraq (2014-2018) Popular Mo...,unknown,1,Islamic State (Iraq),unknown,2,12,Middle East,Iraq,Anbar,Anah,unknown,Aradi as Sakrah,34.1915,42.0313,3,6,1567462133,46,56,1,0.0
2,2017-01-01,2017,Explosions/Remote violence,Air/drone strike,Military Forces of Iraq (2014-2018),unknown,1,Islamic State (Iraq),unknown,2,12,Middle East,Iraq,Ninewa,Mosul,unknown,Mosul,36.3350,43.1189,1,20,1572403769,46,56,1,0.0
3,2017-01-01,2017,Explosions/Remote violence,Remote explosive/landmine/IED,Islamic State (Iraq),unknown,2,Civilians (Iraq),unknown,7,27,Middle East,Iraq,Najaf,Najaf,unknown,Najaf,32.0259,44.3462,2,6,1572403770,46,56,1,0.0
4,2017-01-01,2017,Battles,Armed clash,Islamic State (Iraq),unknown,2,Police Forces of Iraq (2014-2018),unknown,1,12,Middle East,Iraq,Najaf,Manathera,unknown,Al Mishkhab,31.8044,44.4893,1,0,1572403770,46,56,1,0.0


In [25]:
# Reading in Economic Data
econ_data = pd.read_csv('data/econ-data-imputed.csv')
econ_data.head()

,Country,Code,Year,Month,Date,Exchange rate USD,Unemployment rate,Rule of law index (-2.5 weak; 2.5 strong),Government effectiveness index (-2.5 weak; 2.5 strong),Control of corruption (-2.5 weak; 2.5 strong),Regulatory quality index (-2.5 weak; 2.5 strong),Voice and accountability index (-2.5 weak; 2.5 strong),Political stability index (-2.5 weak; 2.5 strong),Corruption Perceptions Index 100 = no corruption,Political rights index 7 (weak) - 1 (strong),Civil liberties index 7 (weak) - 1 (strong),Short-term political risk (1=low 7=high),Medium/long-term political risk (1=low 7=high),Political violence risk (1=low 7=high),Mobile phone subscribers per 100 people,Property rights index (0-100),Freedom from corruption index (0-100),Business freedom index (0-100),Labor freedom index (0-100),Life expectancy in years,Fragile state index 0 (low) - 120 (high),Security threats index 0 (low) - 10 (high),State legitimacy index 0 (high) - 10 (low),Public services index 0 (high) - 10 (low),Human rights and rule of law index 0 (high) - 10 (low),Refugees and displaced persons index 0 (low) - 10 (high),Labor force million people,Labor force participation rate,Percent urban population,Population density people per square km
0,Egypt,EGY,2017,1,1/1/17,18.6331,11.74,-0.54,-0.62,-0.55,-0.86,-1.25,-1.42,32,6,5,4,6,5,106.76,35,33,67,51,71.66,89.8,8.1,8.2,4.9,9.8,7.3,30.66,48.01,42.71,97.0
1,Egypt,EGY,2017,2,2/1/17,17.2295,11.74,-0.54,-0.62,-0.55,-0.86,-1.25,-1.42,32,6,5,4,6,5,106.76,35,33,67,51,71.66,89.8,8.1,8.2,4.9,9.8,7.3,30.66,48.01,42.71,97.0
2,Egypt,EGY,2017,3,3/1/17,17.6674,11.74,-0.54,-0.62,-0.55,-0.86,-1.25,-1.42,32,6,5,4,6,5,106.76,35,33,67,51,71.66,89.8,8.1,8.2,4.9,9.8,7.3,30.66,48.01,42.71,97.0
3,Egypt,EGY,2017,4,4/1/17,18.0983,11.74,-0.54,-0.62,-0.55,-0.86,-1.25,-1.42,32,6,5,4,6,5,106.76,35,33,67,51,71.66,89.8,8.1,8.2,4.9,9.8,7.3,30.66,48.01,42.71,97.0
4,Egypt,EGY,2017,5,5/1/17,18.0884,11.74,-0.54,-0.62,-0.55,-0.86,-1.25,-1.42,32,6,5,4,6,5,106.76,35,33,67,51,71.66,89.8,8.1,8.2,4.9,9.8,7.3,30.66,48.01,42.71,97.0


In [26]:
econ_data['Date'] = pd.to_datetime(econ_data['Date'])

In [27]:
conflict = pd.merge(conflict, econ_data,
                            how='left',
                            left_on=['country', 'year', 'month'],
                            right_on=['Country', 'Year', 'Month'])

In [28]:
conflict.isnull().sum()

event_date                                                  0
year                                                        0
event_type                                                  0
sub_event_type                                              0
actor1                                                      0
assoc_actor_1                                               0
inter1                                                      0
actor2                                                      0
assoc_actor_2                                               0
inter2                                                      0
interaction                                                 0
region                                                      0
country                                                     0
admin1                                                      0
admin2                                                      0
admin3                                                      0
location

In [29]:
conflict.drop(columns=['Country',
                       'Code',
                       'Year',
                       'Month',
                       'Date'
                       ],
                       inplace=True)

In [30]:
# dummies = pd.get_dummies(conflict[['event_type', 'country', 'admin1']])

In [31]:
# conflict = pd.concat([conflict, dummies], axis=1)

In [31]:
conflict.to_csv('data/conflict-econ-tor.csv', index=False)